## 1. Cargar y visualizar los datos

In [ ]:
import pandas as pd
# Cargar el archivo CSV
archivo = "valencia_clima_unificado.csv"
df = pd.read_csv(archivo)

# Ver las primeras y últimas filas
display(df.head())
display(df.tail())







### **1. Estructura y tamaño del dataset**  
- El dataset tiene **2561 filas** y **33 columnas**, lo que sugiere un conjunto de datos moderadamente grande para análisis climáticos.  
- El tipo de datos es mayoritariamente **numérico (float64 e int64)**, con algunas columnas categóricas **(object)**.  

### **2. Columnas de interés**  
- Variables principales: `tempmax`, `tempmin`, `temp`, `humidity`, `precip`, `windspeed`, etc.  
- Variables categóricas: `conditions`, `description`, `icon` (pueden ser útiles para análisis de patrones meteorológicos).  
- `datetime`: Está en formato **object**, sería recomendable convertirla a formato `datetime64` para facilitar análisis temporales.  
- `sunrise` y `sunset` también están en formato `object`, podrían convertirse a `datetime` para cálculos solares.  

### **3. Datos faltantes**  
- `preciptype` tiene **1857 valores nulos** (~72%), lo que podría indicar que la mayoría de los días no hubo precipitación o que los datos no están bien registrados.  
- `severerisk` tiene **1829 valores nulos**, podría significar que la mayoría de los días no hubo eventos meteorológicos extremos.  
- `stations` tiene 2 valores nulos, probablemente no es un problema grave.  

### **4. Posibles problemas y limpieza de datos**  
- **Valores faltantes:** Se debe decidir si imputar valores en `preciptype` o eliminar la columna si no es relevante.  
- **Conversión de datos:** Convertir `datetime`, `sunrise`, `sunset` a formato adecuado.  
- **Verificación de valores extremos:**  
  - `tempmax`, `tempmin`, `humidity`, `precip` pueden contener valores fuera de rangos esperados.  
  - `windspeed` y `windgust` pueden ser analizados para detectar posibles errores en la medición.  


In [ ]:
import pandas as pd

# Obtener tipos de variables
variable_types = df.dtypes

# Función para clasificar tipo de variable
def classify_type(dtype):
    if pd.api.types.is_numeric_dtype(dtype):
        return "Numérica"
    elif pd.api.types.is_bool_dtype(dtype):
        return "Binaria"
    else:
        return "Categórica"

# Crear DataFrame con la información
variable_info = pd.DataFrame({
    "Variable": df.columns,
    "Tipo": [classify_type(dtype) for dtype in variable_types],
})

# Diccionario con descripciones
descriptions = {
    "name": "Nombre de la ciudad",
    "datetime": "Fecha del registro",
    "tempmax": "Temperatura máxima del día (°C)",
    "tempmin": "Temperatura mínima del día (°C)",
    "temp": "Temperatura promedio del día (°C)",
    "feelslikemax": "Sensación térmica máxima (°C)",
    "feelslikemin": "Sensación térmica mínima (°C)",
    "feelslike": "Sensación térmica promedio (°C)",
    "dew": "Punto de rocío (°C)",
    "humidity": "Humedad relativa (%)",
    "precip": "Precipitación total (mm)",
    "precipprob": "Probabilidad de precipitación (%)",
    "precipcover": "Cobertura de precipitación (%)",
    "preciptype": "Tipo de precipitación (lluvia, nieve, etc.)",
    "snow": "Cantidad de nieve (cm)",
    "snowdepth": "Profundidad de nieve (cm)",
    "windgust": "Ráfaga de viento máxima (km/h)",
    "windspeed": "Velocidad promedio del viento (km/h)",
    "winddir": "Dirección del viento (grados)",
    "sealevelpressure": "Presión atmosférica al nivel del mar (hPa)",
    "cloudcover": "Cobertura de nubes (%)",
    "visibility": "Visibilidad (km)",
    "solarradiation": "Radiación solar (W/m²)",
    "solarenergy": "Energía solar acumulada (MJ/m²)",
    "uvindex": "Índice UV máximo",
    "severerisk": "Riesgo de condiciones climáticas severas",
    "sunrise": "Hora del amanecer",
    "sunset": "Hora del atardecer",
    "moonphase": "Fase lunar (0: Nueva - 1: Llena)",
    "conditions": "Condiciones climáticas descriptivas",
    "description": "Descripción detallada del clima",
    "icon": "Ícono representativo del clima",
    "stations": "Estaciones meteorológicas utilizadas"
}

# Asignar descripciones, dejando "Descripción pendiente" si no está en el diccionario
variable_info["Descripción"] = variable_info["Variable"].map(descriptions).fillna("Descripción pendiente")

# Mostrar la tabla
from tabulate import tabulate
print(tabulate(variable_info, headers="keys", tablefmt="fancy_grid"))

## 2. Identificar valores nulos y duplicados

In [ ]:

# Identificar valores nulos
valores_nulos = df.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos)

# Revisar si hay filas o columnas con muchos datos faltantes
# Por ejemplo, si más del 50% de los datos de una columna están faltando:
columnas_faltantes = df.columns[df.isnull().mean() > 0.5]
print(f"\nColumnas con más del 50% de datos faltantes: {list(columnas_faltantes)}")

# Comprobar filas duplicadas
filas_duplicadas = df.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {filas_duplicadas}")

In [ ]:
# Verificar si hay fechas duplicadas en la columna 'datetime'
fechas_duplicadas = df['datetime'].duplicated().sum()
print(f"\nNúmero de fechas duplicadas en la columna 'datetime': {fechas_duplicadas}")


### Tratamiento de datos duplicados

In [ ]:
# Eliminar las filas duplicadas en la columna 'datetime'
df_sin_duplicados = df.drop_duplicates(subset='datetime')

# Verificar nuevamente el número de fechas duplicadas
fechas_duplicadas_post = df_sin_duplicados['datetime'].duplicated().sum()
print(f"\nNúmero de fechas duplicadas después de eliminar: {fechas_duplicadas_post}")

# Texto explicativo para más adelante
texto_explicativo = """
Se ha identificado que existen 3 fechas duplicadas en la columna 'datetime'. 
Estas filas duplicadas han sido eliminadas del conjunto de datos.
Mas adelante veremos que hacemos con la columna que tiene tantos valores  faltantes, consideraremos si es mejor eliminar la columna o que hacer
"""
print(texto_explicativo)


### Tratamiento de valores nulos

  

#### 📌 `preciptype` (Tipo de precipitación, 1857 nulos)  
- **Posible causa**: Días sin precipitación probablemente tengan `preciptype` nulo porque no hubo lluvia o nieve.  
- **Solución recomendada**:  
  - Si `precip == 0`, rellenar con `"Ninguna"` o `"Sin precipitación"`.  
  - Si `precip > 0`, revisar si podemos inferir el tipo basado en temperatura (`temp`):  
    - Si `temp < 0°C`, probablemente nieve.  
    - Si `temp > 0°C`, probablemente lluvia.  
  - Si no se puede inferir, imputar con `"Desconocido"` o eliminar si la variable es poco relevante.  

---

#### 📌 `severerisk` (Riesgo severo, 1829 nulos)  
- **Posible causa**: Si los días sin datos son días sin eventos severos, el nulo puede significar "sin riesgo".  
- **Solución recomendada**:  
  - Rellenar con `0` si la variable es categórica (donde `0 = sin riesgo`).    

---

#### 📌 `stations` (Estaciones meteorológicas, 2 nulos)  
- **Posible causa**: Puede ser un error en la captura de datos.  
- **Solución recomendada**:  
  - Revisar si los valores se pueden recuperar de registros cercanos.  
  - Si no es crítico, eliminar las filas (solo son 2).  

In [ ]:
import pandas as pd

# Cargar datos (suponiendo que están en un DataFrame llamado df)

# Imputar preciptype
df.loc[df['precip'] == 0, 'preciptype'] = 'Ninguna'  
df['preciptype'].fillna('Desconocido', inplace=True)

# Imputar severerisk con 0 (asumiendo que 0 significa "sin riesgo")
df['severerisk'].fillna(0, inplace=True)

# Eliminar filas con estaciones nulas (solo 2 registros)
df.dropna(subset=['stations'], inplace=True)


In [ ]:

# Identificar valores nulos
valores_nulos = df.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos)

## 3. Análisis de distribuciones y estadísticas descriptivas

In [ ]:
# Obtener estadísticas descriptivas
estadisticas = df.describe()
print("Estadísticas descriptivas:")
(estadisticas)



### **1. Información general**  
- **Número de observaciones:** Todas las variables tienen **2561 datos**, excepto `severerisk` (732 datos), lo que indica una gran cantidad de valores nulos en esta variable.  
- **Media y desviación estándar:** La mayoría de las variables tienen un comportamiento normal con valores dentro de un rango esperado, pero algunas presentan valores extremos.  

### **2. Temperaturas** (`tempmax`, `tempmin`, `temp`, `feelslikemax`, `feelslikemin`, `feelslike`)  
- **Máxima registrada:** 44.9°C  
- **Mínima registrada:** -2.2°C  
- **Rango típico:** Entre 6°C y 28.6°C para temperaturas máximas y -2.2°C a 28.1°C para mínimas.  
- **Sensación térmica (`feelslike`)** sigue una distribución similar a la temperatura real.  
- **Posibles valores extremos:** Los valores de `feelslikemin` (-4.9°C) y `feelslikemax` (45.9°C) deberían revisarse en contexto con la humedad y el viento.  

### **3. Humedad y punto de rocío (`humidity`, `dew`)**  
- **Humedad promedio:** 63.85%  
- **Rango:** 26.4% (mínimo) a 95% (máximo), lo que sugiere diferencias climáticas significativas.  
- **Punto de rocío (`dew`)** varía de -10.9°C a 23.6°C, con una media de 10.66°C.  

### **4. Precipitación (`precip`, `precipprob`, `precipcover`, `snow`, `snowdepth`)**  
- **Precipitación media:** 1.61 mm, pero la mediana es **0 mm**, lo que indica que muchos días no llueve.  
- **Máxima precipitación:** 176.39 mm en un día, lo que sugiere eventos extremos.  
- **Probabilidad de precipitación (`precipprob`)** varía entre 0% y 100%.  
- **Cobertura de precipitación (`precipcover`)** tiene una mediana de **0%**, lo que confirma que en la mayoría de los días no hay lluvia.  
- **Nieve (`snow`, `snowdepth`)** es prácticamente inexistente, con valores máximos de 0.7 mm y 3.1 mm respectivamente, lo que sugiere que el área analizada no suele experimentar nevadas significativas.  

### **5. Viento (`windgust`, `windspeed`, `winddir`)**  
- **Rachas máximas de viento (`windgust`)** alcanzan **179 km/h**, lo que parece ser un valor extremo que podría requerir revisión.  
- **Velocidad media del viento (`windspeed`)** es **14.8 km/h**, con un máximo de **58.7 km/h**, lo que indica días con condiciones ventosas.  
- **Dirección del viento (`winddir`)** varía entre **0.2° y 359.6°**, cubriendo todo el espectro direccional.  

### **6. Presión atmosférica (`sealevelpressure`)**  
- **Media:** 1017.17 hPa  
- **Rango:** 992.9 hPa a 1038.3 hPa  
- **Valores normales:** Se encuentra dentro de los rangos típicos de la presión atmosférica a nivel del mar, con algunas variaciones que pueden estar asociadas a sistemas meteorológicos como frentes y tormentas.  

### **7. Cobertura de nubes y visibilidad (`cloudcover`, `visibility`)**  
- **Cobertura nubosa media:** 36.77%  
- **Máxima:** 96.3%, lo que indica días muy nublados.  
- **Visibilidad media:** 8.85 km, con un rango de **3.1 km a 23.7 km**, lo que indica que en algunos días la visibilidad puede estar muy reducida, posiblemente debido a niebla, lluvia intensa o contaminación.  

### **8. Radiación y energía solar (`solarradiation`, `solarenergy`, `uvindex`)**  
- **Radiación solar media:** 195.51 W/m², con un máximo de **359.8 W/m²**.  
- **Energía solar media:** 16.88 MJ/m².  
- **Índice UV medio:** 7.05, con un máximo de **10**, lo que indica días con alta exposición solar.  

### **9. Riesgo severo y fase lunar (`severerisk`, `moonphase`)**  
- **Riesgo severo (`severerisk`)** tiene muchos valores nulos, lo que sugiere que podría no estar bien documentado.  
- **Fase lunar (`moonphase`)** tiene un rango de **0 a 0.98**, indicando que se ha registrado en diferentes etapas del ciclo lunar.  

### **Conclusiones generales**  
- **El clima general parece templado**, con temperaturas medias de 18.4°C y una humedad media de 63.85%.  
- **La precipitación es poco frecuente**, con muchos días sin lluvia, aunque hay eventos extremos de más de 100 mm en un solo día.  
- **El viento tiene algunos valores extremos** en rachas de hasta 179 km/h, lo que puede ser un error o un evento meteorológico significativo.  
- **Los datos de riesgo severo son incompletos**, lo que limita su análisis.  
- **El índice UV puede alcanzar niveles peligrosos**, lo que sugiere que en ciertos días es necesario protección solar.


## 4.Distribución de los datos y anomalías

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil

# Crear carpeta para guardar los histogramas
os.makedirs("histogramas", exist_ok=True)

# Seleccionar columnas numéricas
numericas = df.select_dtypes(include=['float64', 'int64']).columns  

# Graficar histogramas y guardarlos como imágenes
for col in numericas:
    plt.figure(figsize=(12, 8))
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Histograma de {col}", fontsize=16)
    plt.xlabel(col, fontsize=14)
    plt.ylabel("Frecuencia", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(alpha=0.3)
    
    # Guardar cada gráfico como imagen en la carpeta histogramas
    ruta = f"histogramas/histograma_{col}.png"
    plt.savefig(ruta, bbox_inches='tight')
    plt.close()

# Comprimir todos los histogramas en un archivo ZIP
shutil.make_archive("histogramas", 'zip', "histogramas")
print("✅ Archivo 'histogramas.zip' creado correctamente.")



### Análisis distribucion variables
#### Cloudcover
 La distribución de "Cloudcover" refleja que en esta base de datos predominan condiciones atmosféricas parcialmente nubladas, con una mayor presencia de valores intermedios de cobertura nubosa y menos días con cielos totalmente despejados o completamente cubiertos.
 #### Dew
 La distribución de "Dew" evidencia que las condiciones atmosféricas más comunes son aquellas con puntos de rocío moderados, presentando ocasionalmente condiciones ligeramente más húmedas (punto de rocío elevado) y, en menor proporción, condiciones secas y frías.
 #### Feelslike
  Muestra claramente dos condiciones térmicas predominantes en esta base de datos, una moderada a fresca y otra más cálida, señalando una posible influencia estacional notable en la sensación térmica promedio.
 #### Feelslikemax
 En resumen, la variable "Feelslikemax" indica claramente que predominan condiciones moderadas a cálidas en cuanto a la sensación térmica máxima diaria, con una presencia ocasional de valores extremos que no constituyen la norma habitual del clima estudiado.
 #### Feelslikemin
 La distribución de la variable Feelslikemin refleja claramente dos patrones diferenciados de condiciones térmicas mínimas percibidas, mostrando una marcada influencia estacional o temporal que genera estos dos escenarios predominantes en la sensación térmica mínima registrada.

#### Humidity
La distribución de la variable Humidity indica que la mayoría de días registrados tienen valores moderadamente altos de humedad relativa, concentrándose principalmente entre un 65% y 75%. Esta concentración revela que las condiciones climáticas de la zona o periodo analizado tienden hacia un ambiente predominantemente húmedo, siendo poco frecuentes los días extremadamente secos o con valores extremos de humedad relativa.
#### Moonphase
La distribución uniforme de la variable Moonphase refleja simplemente la regularidad natural del ciclo lunar, indicando que el conjunto de datos no presenta ningún sesgo o irregularidad en cuanto a la fase lunar observada en las fechas registradas. Esto es justamente lo esperable en este tipo de variable y confirma la calidad y representatividad temporal de los datos en lo que respecta al fenómeno lunar.
#### Precip
 La variable Precip indica claramente que en la región o periodo estudiado predominan los días sin precipitación o con precipitaciones muy leves, siendo relativamente escasos pero importantes los eventos aislados de lluvia fuerte o intensa.
 #### Precipcover
La distribución de Precipcover refleja claramente que, aunque las precipitaciones ocurren con cierta frecuencia, tienden a cubrir áreas limitadas o específicas, siendo poco frecuentes los días con cobertura muy extendida o generalizada. 
#### Precipprob
Precipprob revela que en esta región o periodo analizado se distingue claramente entre días secos sin posibilidad de lluvia y días húmedos con lluvias seguras, siendo poco habitual encontrar probabilidades intermedias o dudosas.
#### Sealevelpressure
La variable Sealevelpressure refleja claramente que las condiciones predominantes en cuanto a presión atmosférica son moderadas y estables, destacando la baja frecuencia de eventos extremos.
#### Severisk
La variable Severerisk muestra claramente que la mayoría de días estudiados son seguros y sin riesgo meteorológico significativo, existiendo solo muy pocos días en los que se identifica riesgo severo moderado o alto.
#### Snow
La distribución de la variable Snow confirma claramente que la caída de nieve es un fenómeno sumamente raro o prácticamente ausente en la región o periodo considerado.
#### Snowdepth
La variable Snowdepth evidencia claramente la ausencia casi total de acumulaciones de nieve, indicando que esta condición climática es extraordinariamente rara en el conjunto de datos.
#### Solarenergy
La distribución de la variable Solarenergy refleja claramente que, aunque predominan días con niveles moderados de energía solar, también hay una considerable cantidad de días con alta disponibilidad solar y, en menor proporción, días con muy poca disponibilidad solar. Esto probablemente refleja una variabilidad climática asociada con las condiciones atmosféricas y estacionales del área estudiada.
#### Solarradiation
la variable Solarradiation muestra claramente la existencia de dos condiciones predominantes: días con radiación moderada y días con radiación solar alta, probablemente relacionados con la presencia o ausencia significativa de nubosidad
#### Temp
 La distribución de Temp indica claramente la existencia de dos tipos predominantes de temperaturas medias en la región o periodo analizado, sugiriendo la marcada presencia de al menos dos temporadas diferentes a lo largo del año.
 #### Tempmax
 La variable Tempmax refleja principalmente condiciones térmicas moderadas y cálidas durante el periodo analizado, con una ligera diferenciación estacional entre rangos específicos de temperaturas máximas diarias.
#### Tempmin
 Tempmin muestra claramente la existencia de dos condiciones predominantes, reflejando la fuerte influencia estacional sobre las temperaturas mínimas diarias observadas.
 #### Uvindex
La distribución del Uvindex refleja claramente que en la región estudiada predominan días con exposición solar intensa, con un índice UV típicamente alto.
#### Visibility
La variable Visibility refleja la existencia clara de dos condiciones predominantes en cuanto a visibilidad atmosférica: una moderada y otra muy buena.
#### Winddir
Winddir revela claramente dos direcciones predominantes y bien definidas del viento en la región analizada.
#### Windgust
Windgust muestra claramente que las ráfagas de viento moderadas predominan, siendo relativamente poco comunes los eventos de ráfagas fuertes o muy fuertes.
#### Windspeed
Windspeed refleja claramente la predominancia de condiciones de viento moderado, siendo escasos los días con vientos muy fuertes.



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Crear carpeta para guardar los boxplots
os.makedirs("boxplots", exist_ok=True)

# Graficar boxplot para cada variable numérica con mayor tamaño
for col in numericas:
    plt.figure(figsize=(12, 8))  # Mayor tamaño para visualizar mejor
    sns.boxplot(x=df[col])
    plt.title(f"Diagrama de caja de {col}", fontsize=16)
    plt.xlabel(col, fontsize=14)
    plt.xticks(fontsize=12)
    plt.grid(alpha=0.3)

    # Guardar gráfico como imagen para mejor visualización
    plt.savefig(f"boxplots/boxplot_{col}.png", bbox_inches='tight')
    plt.show()




### 1. **Cloudcover (Cobertura Nubosa)**:
- **Anomalías:** Se observa un valor anómalo alto (cercano al 100%), que está fuera del rango típico, pero muy cerca del límite superior.
- **Evaluación:** Este valor podría corresponder a un día excepcionalmente nublado, pero es completamente plausible en términos reales.
- **Recomendación:** Mantener esta anomalía, ya que refleja condiciones naturales posibles, aunque poco frecuentes.

---

### 2. **Dew (Punto de rocío)**:
- **Anomalías:** Se observan algunas anomalías en valores muy bajos (negativos cercanos a -10 °C).
- **Evaluación:** Estos valores extremos pueden indicar condiciones muy secas o frías que pueden ocurrir en días puntuales, aunque podrían requerir una validación adicional para confirmar su autenticidad.
- **Recomendación:** Revisar manualmente estos datos para asegurar que no sean errores. Si los valores son reales (algo plausible), mantenerlos, de lo contrario, imputarlos o eliminarlos. 
Vamos a dejar las anomalias ya que revisando los datos manualmente nos hemos dado cuenta que la anomalias son de diass donde las temperaturas son muy bajas por lo que no es tan raro que el punto de rocio fuera bajo

---

### 3. **Feelslike (Sensación térmica promedio)**:
- **Anomalías:** No se observan anomalías significativas.
- **Evaluación:** Todos los datos están dentro de rangos normales, sin valores anómalos destacados.
- **Recomendación:** No es necesario realizar ningún tratamiento adicional.

---

### 4. **Feelslikemax (Sensación térmica máxima)**:
- **Anomalías:** Se observa un único valor anómalo alto alrededor de 43-44 °C.
- **Evaluación:** Este valor podría corresponder a un día excepcionalmente caluroso. Aunque elevado, es posible en ciertas condiciones climáticas específicas.
- **Recomendación:** Mantener esta anomalía si se confirma que es un dato real, ya que refleja eventos climáticos extremos reales y posibles.
No la vamos a quitar ya que observando todas las variables de ese dia vemos que es un dia que hizo mucho calor, la temperatura maxima fue muy alta por lo que ese dia fue un caso extremo pero no es que sea esta variable un caso expecional

---

### 5. **Feelslikemin (Sensación térmica mínima)**:
- **Anomalías:** No se observan anomalías en el diagrama de caja.
- **Evaluación:** Los valores están bien distribuidos y no presentan observaciones extremas fuera de lo esperado.
- **Recomendación:** No requiere acciones especiales.

---


### 6. **Humidity (Humedad)**:
- **Anomalías:** Se observa una anomalía leve hacia la izquierda, con valores muy bajos de humedad (inferior al 30%).
- **Evaluación:** Podría representar condiciones atmosféricas muy secas que, aunque raras, son posibles.
- **Recomendación:** Mantener tras validar manualmente que estos valores sean reales. Si hay errores evidentes, corregirlos o imputarlos. 
 Aunque no imposible, el valor es demasiado extremo y parece un error junto con el valor anómalo de punto de rocío.Por lo que voy a imputarlo y ya
---

### 7. **Moonphase (Fase Lunar)**:
- **Anomalías:** No se observan anomalías significativas.
- **Evaluación:** La variable tiene un comportamiento regular y esperado.
- **Recomendación:** No requiere tratamiento especial.

---

### 8. **Precip (Precipitación)**:
- **Anomalías:** Se observan numerosas anomalías, especialmente hacia valores altos de precipitación (algunos valores por encima de 50 mm e incluso uno extremadamente alto alrededor de 175 mm).
- **Evaluación:** Estos valores extremos son posibles, pero muy poco frecuentes. Probablemente corresponden a eventos puntuales de lluvia intensa.
- **Recomendación:** 
  - Mantener estos valores, ya que representan eventos climáticos extremos, importantes para ciertos análisis meteorológicos.
  - Realizar una validación adicional solo en el valor extremo (175 mm) para confirmar su autenticidad.
El valor de esta precipitación en un día con estas condiciones (cielo muy cubierto, alta probabilidad de lluvia y baja presión atmosférica) es absolutamente plausible y coherente.

---

### 9. **Precipcover (Cobertura de Precipitación)**:
- **Anomalías:** Se observan muchos valores extremos (por encima del 20%), especialmente hacia el lado derecho hasta alcanzar el 100%.
- **Evaluación:** Son valores esperables, ya que pueden existir días en los que la precipitación es muy generalizada.
- **Recomendación:** Mantener estas anomalías, pues reflejan condiciones climáticas posibles (lluvias generalizadas). Confirmar solo puntualmente si hay registros claramente erróneos (100% cobertura repetidamente podría requerir revisión).

---

### 10. **Precipprob (Probabilidad de Precipitación)**:
- **Anomalías:** No se observan anomalías.
- **Evaluación:** El rango es esperado y sin observaciones fuera de lo normal.
- **Recomendación:** No requiere acciones especiales.

---

### 11. **Sealevelpressure (Presión atmosférica al nivel del mar)**:
- **Anomalías:** Muchas anomalías hacia ambos lados (muy bajas cerca de 1000 hPa y muy altas cerca de 1035 hPa).
- **Evaluación:** Son valores extremos reales y posibles, aunque poco frecuentes, asociados normalmente a condiciones meteorológicas excepcionales (tormentas, anticiclones).
- **Recomendación:** Mantener estos valores por representar condiciones atmosféricas reales, pero considerar revisarlos puntualmente solo si existen valores extremadamente atípicos o muy frecuentes en los extremos. Lo dejamos igual si tienen sentido

---

### 12. **Severerisk (Riesgo meteorológico severo)**:
- **Anomalías:** Observaciones dispersas con valores altos (cercanos o iguales a 100%).
- **Evaluación:** Estas anomalías representan días puntuales con condiciones meteorológicas muy adversas o peligrosas.
- **Recomendación:** Mantener claramente estas observaciones porque son muy relevantes para el análisis meteorológico de eventos extremos.

---

### 13. **Snow (Nieve caída)**:
- **Anomalías:** Solo se observa un valor extremo relativamente alto (aproximadamente 0.7 mm).
- **Evaluación:** Este valor podría representar un evento excepcional o puntual de nieve.
- **Recomendación:** Revisar este valor para confirmar que es real. Si no es plausible (dependiendo del clima local estudiado), imputarlo o eliminarlo. Lo dejamos ya que no es tan alto si que es verdad que en valencia nunca nieva y que se presente un dia de nieve es raro pero no tanto

---

### 14. **Snowdepth (Profundidad de nieve acumulada)**:
- **Anomalías:** Observaciones aisladas, con valores bajos (cercanos a 0.2-0.3 mm) y otras anomalías con valores más altos (alrededor de 3 mm).
- **Evaluación:** Son valores posiblemente reales, pero raros. Pueden corresponder a eventos aislados.
- **Recomendación:** Realizar revisión manual de estos registros para decidir si mantenerlos. Si el clima de la zona normalmente no incluye nieve, eliminar o imputar estos valores. Igual que en la de antes

---


### 15. **Solarenergy (Energía solar)**:
- **Anomalías:** No se observan anomalías destacadas en el diagrama.
- **Evaluación:** Valores esperados y dentro del rango normal.
- **Recomendación:** No requiere ninguna acción especial.

---

### 16. **Solarradiation (Radiación solar)**:
- **Anomalías:** No presenta anomalías visibles.
- **Evaluación:** Distribución dentro de rangos normales esperados.
- **Recomendación:** No requiere tratamiento especial.

---

### 17. **Temp (Temperatura promedio)**:
- **Anomalías:** No presenta anomalías en el diagrama de caja.
- **Evaluación:** Valores completamente normales dentro del rango típico.
- **Recomendación:** No es necesario realizar ninguna acción adicional.

---

### 18. **Tempmax (Temperatura máxima)**:
- **Anomalías:** Se observa una sola anomalía, con un valor alto alrededor de 44-45 °C.
- **Evaluación:** Este valor extremo podría representar condiciones de calor extremo puntuales pero reales, especialmente en olas de calor.
- **Recomendación:** Mantener tras verificar brevemente su autenticidad, pues refleja eventos reales y posibles. Igual que en la de temp

---

### 19. **Tempmin (Temperatura mínima)**:
- **Anomalías:** No presenta anomalías visibles.
- **Evaluación:** Valores dentro de lo normal y esperable para esta variable.
- **Recomendación:** No requiere tratamiento adicional.

---

### 20. **Uvindex (Índice UV)**:
- **Anomalías:** No presenta anomalías en el diagrama de caja.
- **Evaluación:** Valores esperados y realistas.
- **Recomendación:** No requiere tratamiento adicional.

---

### 21. **Visibility (Visibilidad)**:
- **Anomalías:** Presenta dos anomalías hacia valores muy altos (alrededor de 20 km de visibilidad).
- **Evaluación:** Podrían corresponder a días excepcionales con condiciones atmosféricas extremadamente claras.
- **Recomendación:** Revisar estos valores puntuales para confirmar su validez, manteniéndolos si son reales, o corrigiéndolos si se confirma error de medición. Los mantenemos ya que tras revisar el resto de variables considero que es un valor raro pero si que puede ser real

---

### 22. **Winddir (Dirección del viento)**:
- **Anomalías:** No presenta anomalías visibles.
- **Evaluación:** Dirección del viento dentro de rangos esperables.
- **Recomendación:** No requiere acciones especiales.

---

### 23. **Windgust (Ráfagas de viento)**:
- **Anomalías:** Numerosas anomalías hacia valores altos (superiores a 50 km/h), incluyendo una observación extremadamente alta (aproximadamente 175 km/h).
- **Evaluación:** Las anomalías reflejan ráfagas de viento fuertes reales y posibles, aunque la más extrema podría requerir validación adicional.
- **Recomendación:** Mantener todas las anomalías, revisando especialmente la más alta para confirmar su autenticidad o corregirla si es un error. La mantenemos

---

### 24. **Windspeed (Velocidad del viento)**:
- **Anomalías:** Varias anomalías hacia valores altos (por encima de 30 km/h, llegando incluso alrededor de 60 km/h).
- **Evaluación:** Estas velocidades podrían ocurrir en condiciones ventosas o en tormentas puntuales.
- **Recomendación:** Mantener estas observaciones al ser eventos reales, revisando únicamente las más extremas si generan dudas. Las mantenemos

---


In [ ]:
#Este trozo de codigo se utiliza para revisar una a una las variables que presentan anomalias, e ha cambiado la variable y se ha ido trasteando
#Calcular límites usando IQR para anomalías (outliers)
Q1 = df['precip'].quantile(0.25)
Q3 = df['precip'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar únicamente las anomalías
anomalías_dew = df[(df['precip'] < limite_inferior) | (df['precip'] > limite_superior)]

# Mostrar anomalías claramente
print(anomalías_dew[['precip']])

# Ver todas las variables en la fila 1977
print(df.loc[2559])



### Imputación de anomalías que no tienen sentido

In [ ]:
mediana_humidity = df['humidity'].median()
df.at[1776, 'humidity'] = mediana_humidity


Finalmente solo hemos imputado un valor ya que es el unico que considero que era un valor muy extraño y raro. El resto los he dejado porque considerandolo con el resto de variables simplemente se trata de un dia especial pero que en conjunto ed todas las variables si que es posible que se de 

## 5. Correlaciones entre variables

In [ ]:
# Calcular la matriz de correlación
correlaciones = df.corr()
print("Matriz de correlación:")
(correlaciones)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Graficar el heatmap de la matriz de correlación
plt.figure(figsize=(20, 10))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.show()


Este diagrama es una **matriz de correlación**, la cual muestra cómo se relacionan entre sí las diferentes variables climáticas de tu base de datos. Aquí están algunos puntos clave que se pueden extraer de la imagen:

### 1. **Relaciones Fuertes (Alto Grado de Correlación Positiva)**
   - **Temperatura**: Las variables relacionadas con la temperatura, como `temp`, `tempmax`, `tempmin`, `feelslikemax`, `feelslikemin`, `feelslike`, muestran correlaciones **muy altas (cercanas a 1)** entre ellas. Esto indica que cuando una de estas aumenta, las demás también tienden a aumentar.
   - **Radiación Solar y Energía Solar**: `solarradiation` y `solarenergy` están fuertemente correlacionadas (~0.95), lo cual es lógico, ya que la energía solar depende de la radiación solar recibida.
   - **Viento**: `windgust` y `windspeed` tienen una correlación alta (~0.87), lo que sugiere que las ráfagas de viento y la velocidad media del viento suelen aumentar juntas.

### 2. **Relaciones Negativas Importantes**
   - **Temperatura y Presión Atmosférica**: `tempmax`, `temp`, `feelslikemax` tienen correlaciones **negativas** con `sealevelpressure` (~-0.27 a -0.39). Esto sugiere que los días más cálidos suelen tener menor presión atmosférica.
   - **Viento y Dirección del Viento**: `winddir` tiene correlaciones **negativas** con `temp` y `feelslike` (~-0.5), lo que podría indicar que ciertas direcciones del viento están asociadas con temperaturas más bajas.

### 3. **Relaciones con la Precipitación**
   - `precip`, `precipprob` y `precipcover` tienen correlaciones entre sí, pero no parecen estar fuertemente correlacionadas con la temperatura, lo que sugiere que la lluvia no está directamente ligada a la temperatura en esta base de datos.
   - `precipcover` y `cloudcover` tienen una **correlación positiva moderada (~0.40)**, lo cual es esperado, ya que la nubosidad alta puede aumentar la probabilidad de lluvia.

### 4. **Factores Relevantes para Eventos Meteorológicos Extremos**
   - `severerisk` tiene correlaciones positivas moderadas con `windspeed` y `precipprob`, lo que indica que eventos severos podrían estar relacionados con días de alta velocidad de viento y probabilidad de lluvia.

